In [1]:
import pywinauto
from pywinauto import clipboard
import time

In [2]:
app = pywinauto.Application().connect(path=r"C:\同花顺软件\同花顺\xiadan.exe",timeout=10)

In [3]:
main = app.top_window()

In [4]:
def get_left_menus_handle():
    while True:
        try:
            handle = main.window(
                control_id=129,
                class_name='SysTreeView32'
            )
            # sometime can't find handle ready, must retry
            handle.wait('ready', 2)
            return handle
        except:
            pass

## 一些message
- 系统正在清算中，请稍后重试

## 查询持有股票

In [7]:
get_left_menus_handle().get_item(['查询[F4]', '资金股票']).click()

In [8]:
from captcha_recognize import captcha_recognize

In [17]:
grid = main.window(control_id=0x417, class_name='CVirtualGridCtrl')

grid.set_focus().right_click()   # 模拟右键
pywinauto.keyboard.SendKeys('c') # 模拟发送C


file_path = "tmp.png"
app.top_window().window(
    control_id=0x965,
    class_name='Static'
).CaptureAsImage().save(file_path)  #保存验证码

captcha_num = captcha_recognize(file_path)  # 识别验证码
print(captcha_num)
app.top_window().window(control_id=0x964, class_name='Edit').set_text(captcha_num)  # 模拟输入验证码

app.top_window().set_focus()
pywinauto.keyboard.SendKeys("{ENTER}")   # 模拟发送enter，点击确定

print(clipboard.GetData())  # 

5827
明细	证券代码	证券名称	股票余额	可用余额	冻结数量	盈亏	成本价	盈亏比例(%)	市价	市值	交易市场	股东帐户	实际数量	可申赎数量	资讯	
	162411	华宝油气	2900	2900	0	-43.55	0.547	-2.75	0.53	1542.80	深圳Ａ股	00101106569	2900	2900		                                                              


In [15]:
import pandas as pd
import io

In [18]:
data = clipboard.GetData()
df = pd.read_csv(io.StringIO(data),
                 delimiter='\t',
                 na_filter=False)
df

,明细,证券代码,证券名称,股票余额,可用余额,冻结数量,盈亏,成本价,盈亏比例(%),市价,市值,交易市场,股东帐户,实际数量,可申赎数量,资讯,Unnamed: 16
0,,162411,华宝油气,2900,2900,0,-43.55,0.547,-2.75,0.53,1542.8,深圳Ａ股,101106569,2900,2900,,


## 模拟买入

In [19]:
get_left_menus_handle().get_item(['买入[F1]']).click()

In [ ]:
def parse_trade_msg(msg):
    pass

In [20]:
stock_id = "162411"

In [21]:
main.window(control_id=0x408, class_name="Edit").set_text(stock_id)  # 设置股票代码

main.window(control_id=0x409, class_name="Edit").set_text("0.547")  # 设置价格

main.window(control_id=0x40A, class_name="Edit").set_text("100")   # 设置股数目

main.window(control_id=0x3EE, class_name="Button").click()   # 点击买入
app.top_window().window(control_id=0x6, class_name='Button').click()  # 确定买入
print(app.top_window().window(control_id=0x3EC, class_name='Static').window_text())
app.top_window().window(control_id=0x2, class_name='Button').click()  # 确定

您的买入委托已成功提交，合同编号：865912566。


## 模拟买出

In [37]:
get_left_menus_handle().get_item(['卖出[F2]']).click()

In [46]:
main.window(control_id=0x408, class_name="Edit").set_text(stock_id)  # 设置股票代码

main.window(control_id=0x409, class_name="Edit").set_text("0.547")  # 设置价格

main.window(control_id=0x40A, class_name="Edit").set_text("100")   # 设置股数目

main.window(control_id=0x3EE, class_name="Button").click()   # 点击卖出

app.top_window().window(control_id=0x6, class_name='Button').click()  # 确定卖出

print(app.top_window().window(control_id=0x3EC, class_name='Static').window_text())

app.top_window().set_focus()
pywinauto.keyboard.SendKeys("{ENTER}")   # 模拟发送enter，点击确定

系统正在清算中，请稍后重试！


## 获取当日委托

In [29]:
get_left_menus_handle().get_item(['查询[F4]', '当日委托']).click()

In [30]:
grid = main.window(
    control_id=0x417,
    class_name='CVirtualGridCtrl'
)
grid.set_focus().right_click()   # 模拟右键
pywinauto.keyboard.SendKeys('c') # 模拟发送C


file_path = "tmp.png"
app.top_window().window(
    control_id=0x965,
    class_name='Static'
).CaptureAsImage().save(file_path)  #保存验证码

captcha_num = captcha_recognize(file_path)  # 识别验证码

app.top_window().window(
    control_id=0x964,
    class_name='Edit'
).set_text(captcha_num)  # 模拟输入验证码

app.top_window().set_focus()
pywinauto.keyboard.SendKeys("{ENTER}")   # 模拟发送enter，点击确定

print(clipboard.GetData())

data = clipboard.GetData()
df = pd.read_csv(io.StringIO(data),
                 delimiter='\t',
                 na_filter=False)
df

备注	操作	证券代码	证券名称	委托数量	成交数量	委托价格	成交均价	撤消数量	委托时间	委托日期	合同编号	
全部成交	买入	162411	华宝油气	100	100	0.547	0.533	0	13:55:17	20180329	865912566	
未成交	卖出	162411	华宝油气	100	0	0.547	0.000	0	14:40:40	20180329	865967836	                                                               


,备注,操作,证券代码,证券名称,委托数量,成交数量,委托价格,成交均价,撤消数量,委托时间,委托日期,合同编号,Unnamed: 12
0,全部成交,买入,162411,华宝油气,100,100,0.547,0.533,0,13:55:17,20180329,865912566,
1,未成交,卖出,162411,华宝油气,100,0,0.547,0.000,0,14:40:40,20180329,865967836,


## 获取当日成交

In [26]:
get_left_menus_handle().get_item(['查询[F4]', '当日成交']).click()

In [28]:
grid = main.window(
    control_id=0x417,
    class_name='CVirtualGridCtrl'
)
grid.set_focus().right_click()   # 模拟右键
pywinauto.keyboard.SendKeys('c') # 模拟发送C


file_path = "tmp.png"
app.top_window().window(
    control_id=0x965,
    class_name='Static'
).CaptureAsImage().save(file_path)  #保存验证码

captcha_num = captcha_recognize(file_path)  # 识别验证码

app.top_window().window(
    control_id=0x964,
    class_name='Edit'
).set_text(captcha_num)  # 模拟输入验证码

app.top_window().set_focus()
pywinauto.keyboard.SendKeys("{ENTER}")   # 模拟发送enter，点击确定

print(clipboard.GetData())

data = clipboard.GetData()
df = pd.read_csv(io.StringIO(data),
                 delimiter='\t',
                 na_filter=False)
df

成交时间	证券代码	证券名称	操作	成交数量	成交均价	成交金额	合同编号	成交编号	撤单数量	股票余额	
13:55:19	162411	华宝油气	买入	100	0.533	53.300	865912566	2339251680	0	3000	                                                


,成交时间,证券代码,证券名称,操作,成交数量,成交均价,成交金额,合同编号,成交编号,撤单数量,股票余额,Unnamed: 11
0,13:55:19,162411,华宝油气,买入,100,0.533,53.3,865912566,2339251680,0,3000,


## 撤单

In [31]:
get_left_menus_handle().get_item(['撤单[F3]']).click()

In [32]:
entrust_no = "865967836"

In [33]:
def get_entrusts():
    grid = main.window(
        control_id=0x417,
        class_name='CVirtualGridCtrl'
    )
    grid.set_focus().right_click()   # 模拟右键
    pywinauto.keyboard.SendKeys('c') # 模拟发送C


    file_path = "tmp.png"
    app.top_window().window(
        control_id=0x965,
        class_name='Static'
    ).CaptureAsImage().save(file_path)  #保存验证码

    captcha_num = captcha_recognize(file_path)  # 识别验证码

    app.top_window().window(
        control_id=0x964,
        class_name='Edit'
    ).set_text(captcha_num)  # 模拟输入验证码

    app.top_window().set_focus()
    pywinauto.keyboard.SendKeys("{ENTER}")   # 模拟发送enter，点击确定

    print(clipboard.GetData())

    data = clipboard.GetData()
    df = pd.read_csv(io.StringIO(data),
                     delimiter='\t',
                     na_filter=False)
    return df.to_dict('records')

entrusts = get_entrusts()

委托时间	证券代码	证券名称	操作	备注	委托数量	成交数量	撤消数量	委托价格	订单类型	成交均价	合同编号	
14:40:40	162411	华宝油气	卖出	未成交	100	0	0	0.547	限价	0.000	865967836	                                                       


In [34]:
entrusts

[{'Unnamed: 12': '',
  '合同编号': 865967836,
  '备注': '未成交',
  '委托价格': 0.547,
  '委托数量': 100,
  '委托时间': '14:40:40',
  '成交均价': 0.0,
  '成交数量': 0,
  '撤消数量': 0,
  '操作': '卖出',
  '订单类型': '限价',
  '证券代码': 162411,
  '证券名称': '华宝油气'}]

In [35]:
def cancel_by_double_click(row):
    x = 50
    y = 30 + 16 * row
    app.top_window().window(
            control_id=1047,
            class_name='CVirtualGridCtrl'
    ).double_click(coords=(x, y))
    
    app.top_window().window(control_id=0x6, class_name='Button').click()  # 确定撤单

    print("msg:", app.top_window().window(control_id=0x3EC, class_name='Static').window_text())
    
    app.top_window().window(control_id=0x2, class_name='Button').click()  # 确定撤单

In [36]:
for i, entrust in enumerate(entrusts):
    print(i, entrust["合同编号"])
    if str(entrust["合同编号"]) == entrust_no:
        print("canceling", entrust_no)
        cancel_by_double_click(i)

0 865967836
canceling 865967836
msg: 您的撤单委托已成功提交，合同编号：865967836。
